In [38]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import random as ran
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# opening data and cleaning it into usable format
with open('in.dta','r') as tsv:
    raw = [line.strip().split('\t') for line in tsv]

split = []
for pt in raw:
    split.append(pt[0].split())
    
in_data = []
for pt in split:
    in_data.append([np.array([float(pt[0]),float(pt[1])]),float(pt[2])])
    
with open('out.dta','r') as tsv:
    raw = [line.strip().split('\t') for line in tsv]

split = []
for pt in raw:
    split.append(pt[0].split())
    
out_data = []
for pt in split:
    out_data.append([np.array([float(pt[0]),float(pt[1])]),float(pt[2])])

In [9]:
# defining phi's
p0 = lambda x1,x2: 1
p1 = lambda x1,x2: x1
p2 = lambda x1,x2: x2
p3 = lambda x1,x2: x1**2
p4 = lambda x1,x2: x2**2
p5 = lambda x1,x2: x1*x2
p6 = lambda x1,x2: np.abs(x1-x2)
p7 = lambda x1,x2: np.abs(x1+x2)
phis=[p0,p1,p2,p3,p4,p5,p6,p7]

In [10]:
def trans(pt, k):
    x = []
    for i in range(k+1):
        x.append(phis[i](pt[0][0],pt[0][1]))
        
    return [np.array(x), pt[1]]

In [6]:
# comparing k values

ks = [3,4,5,6,7]
for k in ks:
    trans_data = []
    for pt in in_data:
        trans_data.append(trans(pt,k))

    train = trans_data[0:25]
    val = trans_data[25:]

    X = []
    y = []
    for pt in train:
        X.append(pt[0])
        y.append(pt[1])

    # standard linear regression
    X = np.array(X)
    y = np.array(y)

    xp = np.linalg.pinv(X)
    w = xp @ y

    e_val = 0
    for pt in val:
        if not np.sign(w.dot(pt[0])) == np.sign(pt[1]):
            e_val += 1
            
    e_out = 0
    for pt in out_data:
        tpt = trans(pt,k)
        if not np.sign(w.dot(tpt[0])) == np.sign(pt[1]):
            e_out += 1

    print("E_out for the transform with k=" + str(k) + " is " + str(float(e_out)/float(len(out_data))))

    print ("E_val for the transform with k=" + str(k) + " is " + str(e_val / float(len(val)))+"\n")

E_out for the transform with k=3 is 0.42
E_val for the transform with k=3 is 0.3

E_out for the transform with k=4 is 0.416
E_val for the transform with k=4 is 0.5

E_out for the transform with k=5 is 0.188
E_val for the transform with k=5 is 0.2

E_out for the transform with k=6 is 0.084
E_val for the transform with k=6 is 0.0

E_out for the transform with k=7 is 0.072
E_val for the transform with k=7 is 0.1



In [26]:
w

array([-0.02336115, -0.09453057,  0.16020727,  0.43355141])

In [7]:
ks = [3,4,5,6,7]
for k in ks:
    trans_data = []
    for pt in in_data:
        trans_data.append(trans(pt,k))

    val = trans_data[0:25]
    train = trans_data[25:]

    X = []
    y = []
    for pt in train:
        X.append(pt[0])
        y.append(pt[1])

    # standard linear regression
    X = np.array(X)
    y = np.array(y)

    xp = np.linalg.pinv(X)
    w = xp @ y

    e_val = 0
    for pt in val:
        if not np.sign(w.dot(pt[0])) == np.sign(pt[1]):
            e_val += 1
            
    e_out = 0
    for pt in out_data:
        tpt = trans(pt,k)
        if not np.sign(w.dot(tpt[0])) == np.sign(pt[1]):
            e_out += 1

    print("E_out for the transform with k=" + str(k) + " is " + str(float(e_out)/float(len(out_data))))

    print ("E_val for the transform with k=" + str(k) + " is " + str(e_val / float(len(val)))+"\n")

E_out for the transform with k=3 is 0.396
E_val for the transform with k=3 is 0.28

E_out for the transform with k=4 is 0.388
E_val for the transform with k=4 is 0.36

E_out for the transform with k=5 is 0.284
E_val for the transform with k=5 is 0.2

E_out for the transform with k=6 is 0.192
E_val for the transform with k=6 is 0.08

E_out for the transform with k=7 is 0.196
E_val for the transform with k=7 is 0.12



In [1]:
from sklearn import svm


In [2]:
def getLine():
    (x1, y1) = (ran.uniform(-1,1), ran.uniform(-1,1))
    (x2, y2) = (ran.uniform(-1,1), ran.uniform(-1,1))
    line = lambda x:(y2-y1)/(x2-x1)*(x-x1)
    return line

In [3]:
def labelPts(N, line):
    pts = []
    
    for i in range(N):
        (x1,x2) = ran.uniform(-1,1), ran.uniform(-1,1)
        
        pts.append([np.array([1,x1,x2,]),np.sign(x2-line(x1))])
    
    return pts

In [55]:
# PLA function

def runPerceptron(data, line):
    w = np.array([0.0,0.0,0.0])
    mis_pts = []
    for pt in data:
        mis_pts.append(pt)

    iters = 0
    while len(mis_pts) is not 0:
        # choose randoim missclasified point and update
        pt = ran.choice(mis_pts)

        w = w + np.multiply(pt[1],pt[0])


        # recheck for misclassified points
        mis_pts = []

        for pt in data:
            if np.sign(w.dot(pt[0])) != pt[1]:
                mis_pts.append(pt)
        iters += 1
    return w

In [227]:
# compare PLA with SVM for N=10
svm_wins = 0
count = 0
while count <1000:
    
    line = getLine()
    data = labelPts(10,line)
    X = []
    y = []
    
    for pt in data:
        X.append(pt[0])
        y.append(pt[1])
        
    
    
    
    if not(not 1.0 in y or not -1.0 in y):
        count += 1
        
        w = runPerceptron(data, line)
        s = svm.SVC(kernel='linear', C=100000000)
        s.fit(X,y)
        
        test_data = labelPts(1000, line)
    
        E_svm = 0
        E_pla = 0
        for pt in test_data:
            if np.sign(w.dot(pt[0])) != pt[1]:
                E_pla += 1

            SVM_guess = s.predict([pt[0]])
            if float(SVM_guess[0]) != np.sign(pt[1]):
                E_svm += 1

        if E_svm < E_pla:
            svm_wins += 1

In [228]:
print ("SVM is better " + str(svm_wins) + " out of 1000 runs")

SVM is better 625 out of 1000 runs


In [231]:
# compare PLA with SVM for N=100
svm_wins = 0
count = 0
svs = []
while count <1000:
    line = getLine()
    data = labelPts(100,line)
    X = []
    y = []
    
    for pt in data:
        X.append(pt[0])
        y.append(pt[1])
        
    
    
    
    if not (not 1.0 in y or not -1.0 in y):
        count += 1
        
        w = runPerceptron(data, line)
        s = svm.SVC(kernel='linear', C=100000000)
        s.fit(X,y)
        svs.append(len(s.support_vectors_))
        
        test_data = labelPts(1000, line)
    
        E_svm = 0
        E_pla = 0
        for pt in test_data:
            if np.sign(w.dot(pt[0])) != pt[1]:
                E_pla += 1

            SVM_guess = s.predict([pt[0]])
            if float(SVM_guess[0]) != np.sign(pt[1]):
                E_svm += 1

        if E_svm < E_pla:
            svm_wins += 1

In [232]:
print ("SVM is better " + str(svm_wins) + " out of 1000 runs")

SVM is better 597 out of 1000 runs


In [233]:
print ("There are an average of " + str(np.mean(svs)) + " support vectors")

There are an average of 2.998 support vectors
